## Grid Search

Now that we know how to evaluate how well a model generalizes, we can take the next step and improve the model’s generalization performance by tuning its parameters. We discussed the parameter settings of many of the algorithms in scikit-learn in Chapters 2 and 3, and it is important to understand what the parameters mean before trying to adjust them. Finding the values of the important parameters of a model (the ones that provide the best generalization performance) is a tricky task, but necessary for almost all models and datasets. 

Because it is such a common task, there are standard methods in scikit-learn to help you with it. The most commonly used method is grid search, which basically means trying all possible combinations of the parameters of interest.

Consider the case of a kernel SVM with an RBF (radial basis function) kernel, as implemented in the *SVC* class. As we discussed in Chapter 2, there are two important parameters: the kernel bandwidth, *gamma*, and the regularization parameter, *C*. 

Say we want to try the values 0.001, 0.01, 0.1, 1, 10, and 100 for the parameter C, and the same for gamma. Because we have six different settings for C and gamma that we want to try, we have 36 combinations of parameters in total. Looking at all possible combinations creates a table (or grid) of parameter settings for the SVM, as shown here:

%%HTML

<table>
    <tr>
      <th>&nbsp;</th>
      <th>C = 0.001</th>
      <th>C = 0.01</th>
      <th>…</th>
      <th>C = 10</th>
    </tr>
    <tr>
      <td>gamma=0.001</td>
	  <td>SVC(C=0.001, gamma=0.001)</td>
	  <td>SVC(C=0.01, gamma=0.001)</td>
      <td>…</td>
	  <td>SVC(C=10, gamma=0.001)</td>
    </tr>
    <tr>
      <td>gamma=0.01</td>
      <td>SVC(C=0.001, gamma=0.01)</td>
      <td>SVC(C=0.01, gamma=0.01)</td>
      <td>…</td>
      <td>SVC(C=10, gamma=0.01)</td>
    </tr>
    <tr>
      <td>…</td>
      <td>…</td>
      <td>…</td>
      <td>…</td>
      <td>…</td>
    </tr>
    <tr>
      <td>gamma=100</td>
      <td>SVC(C=0.001, gamma=100)</td>
      <td>SVC(C=0.01, gamma=100)</td>
      <td>…</td>
      <td>SVC(C=10, gamma=100)</td>
  </tr>